### Data Wrangling

https://docs.google.com/presentation/d/1v54Tr4POZj9K4zsaHOn7U22rzdSlTXsyCgE5lfhd-rA/edit?usp=sharing

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.getOrCreate()

#### Read data 

We will use San Antonio's 311 call data for this lesson and exercises. Dowload the CSV data from Google classroom

https://classroom.google.com/u/0/w/Mzg3MTg5NzU1Njk1/tc/Mzg3MTg5NzU1NzE1

In [2]:
# Read in CSV file 
source = (spark.read.csv("source.csv",
                     sep=",",
                     header=True,
                     inferSchema=True)
     )

In [3]:
# Another way to read in data:

(
    spark.read.format("csv")
    .option("sep", ",")
    .option("inferSchema", True)
    .option("header", True)
    .load("source.csv")
)

DataFrame[source_id: string, source_username: string]

#### Data Schemas
Spark includes a concept of a data schema  
Specify the types of our data ahead of time

In [4]:
from pyspark.sql.types import StructType, StructField, StringType

schema = StructType(
    [
        StructField("source_id", StringType()),
        StructField("source_username", StringType()),
    ]
)


# Read csv, but now we specify the schema:

source = spark.read.csv("source.csv", header=True, schema=schema)

In [5]:
source.printSchema()

root
 |-- source_id: string (nullable = true)
 |-- source_username: string (nullable = true)



#### Writing data

In [6]:
# write data to a destination using .write property

# source.write.json("source_json.json", mode="overwrite")

#### Data Preparation

In [7]:
# Read the case.csv file

df = spark.read.csv("case.csv", header=True, inferSchema=True)

In [8]:
# shape of dataframe
len(df.columns), df.count() 

(14, 841704)

In [9]:
# look at first three records
df.show(3, vertical=True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 1/1/18 0:42          
 case_closed_date     | 1/1/18 12:29         
 SLA_due_date         | 9/26/20 0:42         
 case_late            | NO                   
 num_days_late        | -998.5087616000001   
 case_closed          | YES                  
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 5                    
-RECORD 1------------------------------------
 case_id              | 1014127333           
 case_opened_date     | 1/1/18 0:46          
 case_closed_date     | 1/3/18 8:11          
 SLA_due_date         | 1/5/18 8:30          
 case_late            | NO                   
 num_days_late        | -2.0126041

In [10]:
#datatypes?
df.dtypes

[('case_id', 'int'),
 ('case_opened_date', 'string'),
 ('case_closed_date', 'string'),
 ('SLA_due_date', 'string'),
 ('case_late', 'string'),
 ('num_days_late', 'double'),
 ('case_closed', 'string'),
 ('dept_division', 'string'),
 ('service_request_type', 'string'),
 ('SLA_days', 'double'),
 ('case_status', 'string'),
 ('source_id', 'string'),
 ('request_address', 'string'),
 ('council_district', 'int')]

**Things to do:**

1. **Rename Columns:**
    - 'SLA_due_date -> case_due_date



2. **Correct Data Types:**
    - case_closed and case_late to boolean
    - council_district as a string
    - case_opened_date, case_closed_date and case_due_date to datetime format


3. **Data Transformation:**
    - request_address: trim and lowercase
    - format council district with leading zeros
    - convert the number of days a case is late to a number of weeks
    
    
4. **New features:**
    - zip_code : extract from address
    - case_age 
    - days_to_closed
    - case_lifetime
    
    
5. **Join cases data with department data:**


#### Rename Columns:

In [11]:
# Rename 'SLA_due_date' to 'case_due_date' using .withColumnRenamed

df = df.withColumnRenamed('SLA_due_date', 'case_due_date')

In [12]:
df.show(1, vertical=True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 1/1/18 0:42          
 case_closed_date     | 1/1/18 12:29         
 case_due_date        | 9/26/20 0:42         
 case_late            | NO                   
 num_days_late        | -998.5087616000001   
 case_closed          | YES                  
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 5                    
only showing top 1 row



#### Correct Data Types:

In [13]:
df.dtypes

[('case_id', 'int'),
 ('case_opened_date', 'string'),
 ('case_closed_date', 'string'),
 ('case_due_date', 'string'),
 ('case_late', 'string'),
 ('num_days_late', 'double'),
 ('case_closed', 'string'),
 ('dept_division', 'string'),
 ('service_request_type', 'string'),
 ('SLA_days', 'double'),
 ('case_status', 'string'),
 ('source_id', 'string'),
 ('request_address', 'string'),
 ('council_district', 'int')]

In [14]:
# correct data types: case_closed and case_late to boolean

df.select("case_closed", "case_late").show(5)

+-----------+---------+
|case_closed|case_late|
+-----------+---------+
|        YES|       NO|
|        YES|       NO|
|        YES|       NO|
|        YES|       NO|
|        YES|      YES|
+-----------+---------+
only showing top 5 rows



In [15]:
# use .withColumn to change columns from string to boolean values

df = df.withColumn("case_closed", expr('case_closed == "YES"'))\
.withColumn("case_late", expr('case_late == "YES"'))

In [16]:
#check the columns
df.select("case_closed", "case_late").show(5)

+-----------+---------+
|case_closed|case_late|
+-----------+---------+
|       true|    false|
|       true|    false|
|       true|    false|
|       true|    false|
|       true|     true|
+-----------+---------+
only showing top 5 rows



In [17]:
# council_district cast as string
df.select('council_district').show(4)

+----------------+
|council_district|
+----------------+
|               5|
|               3|
|               3|
|               3|
+----------------+
only showing top 4 rows



In [18]:
# council_district as a string instead of int
# df.withColumn("council_district",col('council_district'), col('council_district').cast('string'))
df = df.withColumn("council_district", col("council_district").cast("string"))


In [19]:
# view the column

df.select('council_district').show(3)

+----------------+
|council_district|
+----------------+
|               5|
|               3|
|               3|
+----------------+
only showing top 3 rows



In [20]:
# check datatypes
df.dtypes

[('case_id', 'int'),
 ('case_opened_date', 'string'),
 ('case_closed_date', 'string'),
 ('case_due_date', 'string'),
 ('case_late', 'boolean'),
 ('num_days_late', 'double'),
 ('case_closed', 'boolean'),
 ('dept_division', 'string'),
 ('service_request_type', 'string'),
 ('SLA_days', 'double'),
 ('case_status', 'string'),
 ('source_id', 'string'),
 ('request_address', 'string'),
 ('council_district', 'string')]

In [21]:
# convert case_opened_date, case_closed_date and case_due_date to datetime format

df.select('case_opened_date', 'case_closed_date', 'case_due_date').show(5)


+----------------+----------------+-------------+
|case_opened_date|case_closed_date|case_due_date|
+----------------+----------------+-------------+
|     1/1/18 0:42|    1/1/18 12:29| 9/26/20 0:42|
|     1/1/18 0:46|     1/3/18 8:11|  1/5/18 8:30|
|     1/1/18 0:48|     1/2/18 7:57|  1/5/18 8:30|
|     1/1/18 1:29|     1/2/18 8:13| 1/17/18 8:30|
|     1/1/18 1:34|    1/1/18 13:29|  1/1/18 4:34|
+----------------+----------------+-------------+
only showing top 5 rows



In [22]:
# to_timestamp, fmt

fmt = "M/d/yy H:mm"

df = (
    df.withColumn("case_opened_date", to_timestamp("case_opened_date", fmt))
    .withColumn("case_closed_date", to_timestamp("case_opened_date", fmt))
    .withColumn("case_due_date", to_timestamp("case_opened_date", fmt))
)

print("--- After")
df.select("case_opened_date", "case_closed_date", "case_due_date").show(5)

--- After
+-------------------+-------------------+-------------------+
|   case_opened_date|   case_closed_date|      case_due_date|
+-------------------+-------------------+-------------------+
|2018-01-01 00:42:00|2018-01-01 00:42:00|2018-01-01 00:42:00|
|2018-01-01 00:46:00|2018-01-01 00:46:00|2018-01-01 00:46:00|
|2018-01-01 00:48:00|2018-01-01 00:48:00|2018-01-01 00:48:00|
|2018-01-01 01:29:00|2018-01-01 01:29:00|2018-01-01 01:29:00|
|2018-01-01 01:34:00|2018-01-01 01:34:00|2018-01-01 01:34:00|
+-------------------+-------------------+-------------------+
only showing top 5 rows



In [23]:
# check the three columns again

# df.select('case_opened_date', 'case_closed_date', 'case_due_date').show(5)

df.select(df.columns[1:4]).show(3)

+-------------------+-------------------+-------------------+
|   case_opened_date|   case_closed_date|      case_due_date|
+-------------------+-------------------+-------------------+
|2018-01-01 00:42:00|2018-01-01 00:42:00|2018-01-01 00:42:00|
|2018-01-01 00:46:00|2018-01-01 00:46:00|2018-01-01 00:46:00|
|2018-01-01 00:48:00|2018-01-01 00:48:00|2018-01-01 00:48:00|
+-------------------+-------------------+-------------------+
only showing top 3 rows



#### Data Transformation

In [24]:
# request_address: trim and lowercase

df.select('request_address').show(5, False)

+-------------------------------------+
|request_address                      |
+-------------------------------------+
|2315  EL PASO ST, San Antonio, 78207 |
|2215  GOLIAD RD, San Antonio, 78223  |
|102  PALFREY ST W, San Antonio, 78223|
|114  LA GARDE ST, San Antonio, 78223 |
|734  CLEARVIEW DR, San Antonio, 78228|
+-------------------------------------+
only showing top 5 rows



In [25]:
# request_address: trim and lowercase

df = df.withColumn("request_address", lower(trim("request_address")))

In [26]:
df.select('request_address').show(3, truncate=False)

+-------------------------------------+
|request_address                      |
+-------------------------------------+
|2315  el paso st, san antonio, 78207 |
|2215  goliad rd, san antonio, 78223  |
|102  palfrey st w, san antonio, 78223|
+-------------------------------------+
only showing top 3 rows



In [27]:
# convert the number of days a case is late to a number of weeks

df = df.withColumn("num_weeks_late", expr("num_days_late / 7"))

In [28]:
df.select('num_days_late','num_weeks_late').show(3)

+-------------------+--------------------+
|      num_days_late|      num_weeks_late|
+-------------------+--------------------+
| -998.5087616000001|        -142.6441088|
|-2.0126041669999997|-0.28751488099999994|
|       -3.022337963|-0.43176256614285713|
+-------------------+--------------------+
only showing top 3 rows



In [29]:
# use format_string function to pad zeros for council_district
df = df.withColumn('council_district',format_string('%03d', col('council_district').cast('int')))

In [30]:
df.select('council_district').show(3)

+----------------+
|council_district|
+----------------+
|             005|
|             003|
|             003|
+----------------+
only showing top 3 rows



#### New features:

In [31]:
# create a new column for zipcode:

df = df.withColumn('zipcode', expr('right(request_address, 5)'))


In [32]:
df.select('zipcode').show(3)

+-------+
|zipcode|
+-------+
|  78207|
|  78223|
|  78223|
+-------+
only showing top 3 rows



In [33]:
# this regex should and does also work
df = df.withColumn('zipcode',regexp_extract('request_address', r'(\d+$)',1))

In [34]:
df.select('zipcode').show(3)

+-------+
|zipcode|
+-------+
|  78207|
|  78223|
|  78223|
+-------+
only showing top 3 rows



case_age: How old the case is; the difference in days between when the case was opened and the current day  
days_to_closed: The number of days between when the case was opened and when it was closed  
case_lifetime: Number of days between when the case was opened and when it was closed, if the case is still open, the number of days since the case was opened  

In [35]:
#create three new columns 'case_age', 'days_to_closed', 'case_lifetime'

df = (
    df.withColumn(
        "case_age", datediff(current_timestamp(), "case_opened_date")
    )
    .withColumn(
        "days_to_closed", datediff("case_closed_date", "case_opened_date")
    )
    .withColumn(
        "case_lifetime",
        when(expr("! case_closed"), col("case_age")).otherwise(   ### "when NOT case_closed, then case_age, otherwise...."
            col("days_to_closed")
        ),
    )
)

In [36]:
df.show(1, False, True)

-RECORD 0----------------------------------------------------
 case_id              | 1014127332                           
 case_opened_date     | 2018-01-01 00:42:00                  
 case_closed_date     | 2018-01-01 00:42:00                  
 case_due_date        | 2018-01-01 00:42:00                  
 case_late            | false                                
 num_days_late        | -998.5087616000001                   
 case_closed          | true                                 
 dept_division        | Field Operations                     
 service_request_type | Stray Animal                         
 SLA_days             | 999.0                                
 case_status          | Closed                               
 source_id            | svcCRMLS                             
 request_address      | 2315  el paso st, san antonio, 78207 
 council_district     | 005                                  
 num_weeks_late       | -142.6441088                         
 zipcode

In [37]:
# read the dept.csv file:

dept = spark.read.csv("dept.csv", header=True, inferSchema=True)
dept.show(5, False, True)

-RECORD 0-----------------------------------------------
 dept_division          | 311 Call Center               
 dept_name              | Customer Service              
 standardized_dept_name | Customer Service              
 dept_subject_to_SLA    | YES                           
-RECORD 1-----------------------------------------------
 dept_division          | Brush                         
 dept_name              | Solid Waste Management        
 standardized_dept_name | Solid Waste                   
 dept_subject_to_SLA    | YES                           
-RECORD 2-----------------------------------------------
 dept_division          | Clean and Green               
 dept_name              | Parks and Recreation          
 standardized_dept_name | Parks & Recreation            
 dept_subject_to_SLA    | YES                           
-RECORD 3-----------------------------------------------
 dept_division          | Clean and Green Natural Areas 
 dept_name              | Parks

In [38]:
# dept.select('dept_division').distinct().show()
# dept.select('dept_name').show()

In [39]:
# join the df and dept dataframe using 'dept_division' as common key
# drop columns as needed (keep standardized_dept_name)
# convert dept_subject_to_SLA to boolean

df = (
    df
    # left join on dept_division
    .join(dept, "dept_division", "left")
    # drop all the columns except for standardized name, as it has much fewer unique values
    .drop(dept.dept_division)
    .drop(dept.dept_name)
    .withColumnRenamed("standardized_dept_name", "department")
    # convert to a boolean
    .withColumn("dept_subject_to_SLA", col("dept_subject_to_SLA") == "YES")
)

In [40]:
df.show(1, False, True)

-RECORD 0----------------------------------------------------
 dept_division        | Field Operations                     
 case_id              | 1014127332                           
 case_opened_date     | 2018-01-01 00:42:00                  
 case_closed_date     | 2018-01-01 00:42:00                  
 case_due_date        | 2018-01-01 00:42:00                  
 case_late            | false                                
 num_days_late        | -998.5087616000001                   
 case_closed          | true                                 
 service_request_type | Stray Animal                         
 SLA_days             | 999.0                                
 case_status          | Closed                               
 source_id            | svcCRMLS                             
 request_address      | 2315  el paso st, san antonio, 78207 
 council_district     | 005                                  
 num_weeks_late       | -142.6441088                         
 zipcode

# Exercises

## Data Acquisition

- These exercises should go in a notebook or script named wrangle. Add, commit, and push your changes.

- This exercises uses the case.csv, dept.csv, and source.csv files from the san antonio 311 call dataset.

### 1. Read the case, department, and source data into their own spark dataframes.

In [41]:
source = (spark.read.csv("source.csv",
                     sep=",",
                     header=True,
                     inferSchema=True)
     )
source.show(3)

+---------+----------------+
|source_id| source_username|
+---------+----------------+
|   100137|Merlene Blodgett|
|   103582|     Carmen Cura|
|   106463| Richard Sanchez|
+---------+----------------+
only showing top 3 rows



In [42]:
source.count(), len(source.columns)

(140, 2)

In [43]:
source.rollup("source_id").count().sort("source_id").show()
# source.where(source['source_id']==127332).show() # there weren't any, try again

+---------+-----+
|source_id|count|
+---------+-----+
|     null|  140|
|   100137|    1|
|   103582|    1|
|   106463|    1|
|   119403|    1|
|   119555|    1|
|   119868|    1|
|   120752|    1|
|   124405|    1|
|   132408|    1|
|   135723|    1|
|   136202|    1|
|   136979|    1|
|   137943|    1|
|   138605|    1|
|   138650|    2|
|   138793|    1|
|   138810|    1|
|   139342|    1|
|   139344|    1|
+---------+-----+
only showing top 20 rows



In [44]:
case = spark.read.csv("case.csv", header=True, inferSchema=True)
case.show(3, vertical=True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 1/1/18 0:42          
 case_closed_date     | 1/1/18 12:29         
 SLA_due_date         | 9/26/20 0:42         
 case_late            | NO                   
 num_days_late        | -998.5087616000001   
 case_closed          | YES                  
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 5                    
-RECORD 1------------------------------------
 case_id              | 1014127333           
 case_opened_date     | 1/1/18 0:46          
 case_closed_date     | 1/3/18 8:11          
 SLA_due_date         | 1/5/18 8:30          
 case_late            | NO                   
 num_days_late        | -2.0126041

In [45]:
case.count(),len(case.columns)

(841704, 14)

In [46]:
dept = spark.read.csv("dept.csv", header=True, inferSchema=True)
dept.show(3)

+---------------+--------------------+----------------------+-------------------+
|  dept_division|           dept_name|standardized_dept_name|dept_subject_to_SLA|
+---------------+--------------------+----------------------+-------------------+
|311 Call Center|    Customer Service|      Customer Service|                YES|
|          Brush|Solid Waste Manag...|           Solid Waste|                YES|
|Clean and Green|Parks and Recreation|    Parks & Recreation|                YES|
+---------------+--------------------+----------------------+-------------------+
only showing top 3 rows



### 2. Let's see how writing to the local disk works in spark:

    - Write the code necessary to store the source data in both csv and json format, store these as sources_csv and sources_json
    - Inspect your folder structure. What do you notice?

In [47]:
source.write.json("sources_json.json", mode="overwrite")
source.write.csv("sources_csv.csv", mode='overwrite')

### 3. Inspect the data in your dataframes. Are the data types appropriate? Write the code necessary to cast the values to the appropriate types.

In [48]:
# source.dtypes # looks good here
# dept.dtypes # the only change I would consider is to make the 'dept_subject_to_SLA' column a boolean
dept = dept.withColumn("dept_subject_to_SLA", expr('dept_subject_to_SLA == "YES"'))
df.select("dept_subject_to_SLA").show(5)
# looking good

+-------------------+
|dept_subject_to_SLA|
+-------------------+
|               true|
|               true|
|               true|
|               true|
|               true|
+-------------------+
only showing top 5 rows



In [49]:
case.dtypes

[('case_id', 'int'),
 ('case_opened_date', 'string'),
 ('case_closed_date', 'string'),
 ('SLA_due_date', 'string'),
 ('case_late', 'string'),
 ('num_days_late', 'double'),
 ('case_closed', 'string'),
 ('dept_division', 'string'),
 ('service_request_type', 'string'),
 ('SLA_days', 'double'),
 ('case_status', 'string'),
 ('source_id', 'string'),
 ('request_address', 'string'),
 ('council_district', 'int')]

In [50]:
case.show(1,vertical=True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 1/1/18 0:42          
 case_closed_date     | 1/1/18 12:29         
 SLA_due_date         | 9/26/20 0:42         
 case_late            | NO                   
 num_days_late        | -998.5087616000001   
 case_closed          | YES                  
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 5                    
only showing top 1 row



In [51]:
case = case.withColumnRenamed('SLA_due_date', 'case_due_date')

In [52]:
# use .withColumn to change columns from string to boolean values
case = case.withColumn("case_closed", expr('case_closed == "YES"'))\
.withColumn("case_late", expr('case_late == "YES"'))

In [53]:
case = case.withColumn("council_district", col("council_district").cast("string"))

# my only comment here is if it wouldn't be better to give the districts more 'categorical sounding' names
# eg 'one','two','three', etc


In [54]:
# to_timestamp, fmt

fmt = "M/d/yy H:mm"

case = (
    case.withColumn("case_opened_date", to_timestamp("case_opened_date", fmt))
    .withColumn("case_closed_date", to_timestamp("case_opened_date", fmt))
    .withColumn("case_due_date", to_timestamp("case_opened_date", fmt))
)

print("--- After")
case.select("case_opened_date", "case_closed_date", "case_due_date").show(5)

--- After
+-------------------+-------------------+-------------------+
|   case_opened_date|   case_closed_date|      case_due_date|
+-------------------+-------------------+-------------------+
|2018-01-01 00:42:00|2018-01-01 00:42:00|2018-01-01 00:42:00|
|2018-01-01 00:46:00|2018-01-01 00:46:00|2018-01-01 00:46:00|
|2018-01-01 00:48:00|2018-01-01 00:48:00|2018-01-01 00:48:00|
|2018-01-01 01:29:00|2018-01-01 01:29:00|2018-01-01 01:29:00|
|2018-01-01 01:34:00|2018-01-01 01:34:00|2018-01-01 01:34:00|
+-------------------+-------------------+-------------------+
only showing top 5 rows



In [55]:
# request_address: trim and lowercase

case = case.withColumn("request_address", lower(trim("request_address")))

In [56]:
# convert the number of days a case is late to a number of weeks

case = case.withColumn("num_weeks_late", expr("num_days_late / 7"))

In [57]:
case = case.withColumn('zipcode',regexp_extract('request_address', r'(\d+$)',1))

In [58]:
#create three new columns 'case_age', 'days_to_closed', 'case_lifetime'

case = (
    case.withColumn(
        "case_age", datediff(current_timestamp(), "case_opened_date")
    )
    .withColumn(
        "days_to_closed", datediff("case_closed_date", "case_opened_date")
    )
    .withColumn(
        "case_lifetime",
        when(expr("! case_closed"), col("case_age")).otherwise(   ### "when NOT case_closed, then case_age, otherwise...."
            col("days_to_closed")
        ),
    )
)

In [59]:
case.show(2, vertical=True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 00:42:00  
 case_due_date        | 2018-01-01 00:42:00  
 case_late            | false                
 num_days_late        | -998.5087616000001   
 case_closed          | true                 
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  el paso st,... 
 council_district     | 5                    
 num_weeks_late       | -142.6441088         
 zipcode              | 78207                
 case_age             | 1506                 
 days_to_closed       | 0                    
 case_lifetime        | 0                    
-RECORD 1------------------------------------
 case_id              | 1014127333

### How old is the latest (in terms of days past SLA) currently open issue? How long has the oldest (in terms of days since opened) currently opened issue been open?

In [60]:
(
case.filter(case['case_closed']=='false')
# .select(col('case_age'))
.withColumn('days_past_SLA', datediff(current_timestamp(),"case_due_date"))
# .show(1,vertical=True)
).select('days_past_SLA').show(1)

# toggling between the commented out code and the .select at the end for different views

+-------------+
|days_past_SLA|
+-------------+
|         1505|
+-------------+
only showing top 1 row



In [61]:
(
case.filter(case['case_closed']=='false')
# .select(col('case_age'))
.select('case_lifetime').tail(3)
)

[Row(case_lifetime=1508), Row(case_lifetime=1507), Row(case_lifetime=1507)]

### How many Stray Animal cases are there?

In [62]:
case.filter(case['service_request_type']=='Stray Animal').count()

26760

### How many service requests that are assigned to the Field Operations department (dept_division) are not classified as "Officer Standby" request type (service_request_type)?

In [63]:
# case.show(1, vertical=True)

In [64]:
(
case.where(col('dept_division')=='Field Operations')
.filter(col('service_request_type')!='Officer Standby').count()
# .show(1, vertical=True)
)

113902

### Convert the council_district column to a string column.

In [65]:
# I did this in an above step

### Extract the year from the case_closed_date column.

In [66]:
# case.show(1, vertical=True)

In [67]:
case = case.withColumn('case_closed_year', year('case_closed_date'))
case.show(1, vertical=True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 00:42:00  
 case_due_date        | 2018-01-01 00:42:00  
 case_late            | false                
 num_days_late        | -998.5087616000001   
 case_closed          | true                 
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  el paso st,... 
 council_district     | 5                    
 num_weeks_late       | -142.6441088         
 zipcode              | 78207                
 case_age             | 1506                 
 days_to_closed       | 0                    
 case_lifetime        | 0                    
 case_closed_year     | 2018                 
only showing top 1 row



### Convert num_days_late from days to hours in new columns num_hours_late.

In [68]:
# case.show(1, vertical=True)

In [69]:
case = case.withColumn('num_hours_late', col('num_days_late') * 24)
case.show(1, vertical=True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 00:42:00  
 case_due_date        | 2018-01-01 00:42:00  
 case_late            | false                
 num_days_late        | -998.5087616000001   
 case_closed          | true                 
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  el paso st,... 
 council_district     | 5                    
 num_weeks_late       | -142.6441088         
 zipcode              | 78207                
 case_age             | 1506                 
 days_to_closed       | 0                    
 case_lifetime        | 0                    
 case_closed_year     | 2018                 
 num_hours_late       | -23964.210

### Join the case data with the source and department data.

In [70]:
dept.show(2)

+---------------+--------------------+----------------------+-------------------+
|  dept_division|           dept_name|standardized_dept_name|dept_subject_to_SLA|
+---------------+--------------------+----------------------+-------------------+
|311 Call Center|    Customer Service|      Customer Service|               true|
|          Brush|Solid Waste Manag...|           Solid Waste|               true|
+---------------+--------------------+----------------------+-------------------+
only showing top 2 rows



In [71]:
source.show(2)

+---------+----------------+
|source_id| source_username|
+---------+----------------+
|   100137|Merlene Blodgett|
|   103582|     Carmen Cura|
+---------+----------------+
only showing top 2 rows



In [72]:
source_counts = source.groupBy("source_id").count().sort("count")
source_counts.show(5)

+---------+-----+
|source_id|count|
+---------+-----+
|   136202|    1|
|   141239|    1|
|  MW16328|    1|
|  df03076|    1|
|   141549|    1|
+---------+-----+
only showing top 5 rows



In [73]:
source_counts.count()

139

In [74]:
case_counts = case.groupBy("source_id").count().sort("count")
case_counts.count()

139

In [75]:
case.show(1, vertical=True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 00:42:00  
 case_due_date        | 2018-01-01 00:42:00  
 case_late            | false                
 num_days_late        | -998.5087616000001   
 case_closed          | true                 
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  el paso st,... 
 council_district     | 5                    
 num_weeks_late       | -142.6441088         
 zipcode              | 78207                
 case_age             | 1506                 
 days_to_closed       | 0                    
 case_lifetime        | 0                    
 case_closed_year     | 2018                 
 num_hours_late       | -23964.210

In [76]:
# join the df and dept dataframe using 'dept_division' as common key
# drop columns as needed (keep standardized_dept_name)
# convert dept_subject_to_SLA to boolean

case = (
    (
    case
    # left join on dept_division
    .join(dept, "dept_division", "left")
    # drop all the columns except for standardized name, as it has much fewer unique values
    .drop(dept.dept_division)
    .drop(dept.dept_name)
    .withColumnRenamed("standardized_dept_name", "department")
    # convert to a boolean
    .withColumn("dept_subject_to_SLA", col("dept_subject_to_SLA") == "YES")
    )
)

In [77]:
case = case.join(source, case.source_id == source.source_id, 'outer').drop(case.source_id)

In [78]:
case.show(1,vertical=True)

-RECORD 0------------------------------------
 dept_division        | Waste Collection     
 case_id              | 1014128056           
 case_opened_date     | 2018-01-02 08:21:00  
 case_closed_date     | 2018-01-02 08:21:00  
 case_due_date        | 2018-01-02 08:21:00  
 case_late            | true                 
 num_days_late        | 5.00681713           
 case_closed          | true                 
 service_request_type | Solid Waste Fees ... 
 SLA_days             | 3.00619213           
 case_status          | Closed               
 request_address      | 3214  stoney fork... 
 council_district     | 10                   
 num_weeks_late       | 0.71525959           
 zipcode              | 78247                
 case_age             | 1505                 
 days_to_closed       | 0                    
 case_lifetime        | 0                    
 case_closed_year     | 2018                 
 num_hours_late       | 120.16361112         
 department           | Solid Wast

### Are there any cases that do not have a request source?

In [79]:
from pyspark.sql.functions import isnan, when, count, col


df_columns = ['source_id','source_username']
case.select([count(when(isnan(c)| col(c).isNull(), c)).alias(c) for c in df_columns]).show()



# i think this is correct, if i have understood the question 

+---------+---------------+
|source_id|source_username|
+---------+---------------+
|        0|              0|
+---------+---------------+



### What are the top 10 service request types in terms of number of requests?

In [80]:
(
case
.groupBy('service_request_type')
.agg(count(col('service_request_type')))
.sort(count(('service_request_type')).desc())
.show(10, vertical=True)
)

-RECORD 0-------------------------------------------
 service_request_type        | No Pickup            
 count(service_request_type) | 89210                
-RECORD 1-------------------------------------------
 service_request_type        | Overgrown Yard/Trash 
 count(service_request_type) | 66403                
-RECORD 2-------------------------------------------
 service_request_type        | Bandit Signs         
 count(service_request_type) | 32968                
-RECORD 3-------------------------------------------
 service_request_type        | Damaged Cart         
 count(service_request_type) | 31163                
-RECORD 4-------------------------------------------
 service_request_type        | Front Or Side Yar... 
 count(service_request_type) | 28920                
-RECORD 5-------------------------------------------
 service_request_type        | Stray Animal         
 count(service_request_type) | 27361                
-RECORD 6-------------------------------------

### What are the top 10 service request types in terms of average days late?

In [81]:
(
case
.groupBy('service_request_type')
.agg(mean(col('num_days_late')))
.sort(mean(('num_days_late')).desc())
.show(10, vertical=True)
)

-RECORD 0------------------------------------
 service_request_type | Zoning: Junk Yards   
 avg(num_days_late)   | 175.95636210420932   
-RECORD 1------------------------------------
 service_request_type | Labeling for Used... 
 avg(num_days_late)   | 162.43032902285717   
-RECORD 2------------------------------------
 service_request_type | Record Keeping of... 
 avg(num_days_late)   | 153.99724039428568   
-RECORD 3------------------------------------
 service_request_type | Signage Requied f... 
 avg(num_days_late)   | 151.63868055333333   
-RECORD 4------------------------------------
 service_request_type | Storage of Used M... 
 avg(num_days_late)   | 142.11255641500003   
-RECORD 5------------------------------------
 service_request_type | Zoning: Recycle Yard 
 avg(num_days_late)   | 135.9285161247979    
-RECORD 6------------------------------------
 service_request_type | Donation Containe... 
 avg(num_days_late)   | 131.75610506358709   
-RECORD 7-------------------------

### Does number of days late depend on department?

In [82]:
(
case
.groupBy('department')
.agg(mean(col('num_days_late')))
.sort(mean(('num_days_late')).desc())
.show(vertical=True)
)

-RECORD 0----------------------------------
 department         | Customer Service     
 avg(num_days_late) | 59.73709149630082    
-RECORD 1----------------------------------
 department         | Solid Waste          
 avg(num_days_late) | -2.200057513672168   
-RECORD 2----------------------------------
 department         | Metro Health         
 avg(num_days_late) | -4.911766979607001   
-RECORD 3----------------------------------
 department         | Parks & Recreation   
 avg(num_days_late) | -5.251521960055156   
-RECORD 4----------------------------------
 department         | Trans & Cap Impro... 
 avg(num_days_late) | -20.612837354052726  
-RECORD 5----------------------------------
 department         | DSD/Code Enforcement 
 avg(num_days_late) | -38.3693889261437    
-RECORD 6----------------------------------
 department         | Animal Care Services 
 avg(num_days_late) | -226.5178394055035   
-RECORD 7----------------------------------
 department         | City Counc

### How do number of days late depend on department and request type?

In [83]:
# case.show(1,vertical=True)

In [84]:
####################################################################
###### This code works great, don't lose it, do build on it
#####################################################################

(
case
.groupBy('department','service_request_type')
.agg(mean(col('num_days_late')))
.sort(mean(('num_days_late')).desc())
.show(10, vertical=True)
)

-RECORD 0------------------------------------
 department           | DSD/Code Enforcement 
 service_request_type | Zoning: Junk Yards   
 avg(num_days_late)   | 175.95636210420932   
-RECORD 1------------------------------------
 department           | DSD/Code Enforcement 
 service_request_type | Labeling for Used... 
 avg(num_days_late)   | 162.43032902285717   
-RECORD 2------------------------------------
 department           | DSD/Code Enforcement 
 service_request_type | Record Keeping of... 
 avg(num_days_late)   | 153.99724039428568   
-RECORD 3------------------------------------
 department           | DSD/Code Enforcement 
 service_request_type | Signage Requied f... 
 avg(num_days_late)   | 151.63868055333333   
-RECORD 4------------------------------------
 department           | DSD/Code Enforcement 
 service_request_type | Storage of Used M... 
 avg(num_days_late)   | 142.11255641500003   
-RECORD 5------------------------------------
 department           | DSD/Code E

In [89]:
# (
# case
# .groupBy('service_request_type')
# .pivot('department')
# .mean('num_days_late')
# # .sort(mean(('num_days_late')).desc())
# .show(vertical=True)#10, vertical=True)
# )


(
case.groupBy('service_request_type')
.pivot('department')
.mean('num_days_late')
# .sort(mean(('num_days_late')).desc())
.show(vertical=True)#10, vertical=True)
)

-RECORD 0----------------------------------------
 service_request_type     | Minimum Housing-O... 
 Animal Care Services     | null                 
 City Council             | null                 
 Customer Service         | null                 
 DSD/Code Enforcement     | -107.00004932180568  
 Metro Health             | null                 
 Parks & Recreation       | null                 
 Solid Waste              | null                 
 Trans & Cap Improvements | null                 
-RECORD 1----------------------------------------
 service_request_type     | Tree Removal         
 Animal Care Services     | null                 
 City Council             | null                 
 Customer Service         | null                 
 DSD/Code Enforcement     | null                 
 Metro Health             | null                 
 Parks & Recreation       | 5.658236274288586    
 Solid Waste              | null                 
 Trans & Cap Improvements | null                 
